In [0]:
GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'

In [0]:
import os, sys

# Allow load project as module
sys.path.insert(0, '../../../..')

In [0]:
%matplotlib inline

In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

# Open stellar grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

In [0]:
fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.wave_lim = [3000, 9000]
grid.preload_arrays = False
grid.load(fn, format='h5')

# Velocity dispersion

In [0]:
from pfs.ga.pfsspec.core.psf import VelocityDispersion

In [0]:
model = grid.get_model(M_H=0, T_eff=4500, log_g=1, a_M=0, )

model.wave.shape, model.flux.shape

In [0]:
np.min(model.flux), np.max(model.flux)

In [0]:
psf = VelocityDispersion(vdisp=1000, reuse_kernel=True)
s = psf.get_optimal_size(model.wave)

In [0]:
cwave, cflux, _, shift = psf.convolve(model.wave, model.flux, size=s, normalize=True)
cwave.shape, cflux.shape, shift, s

In [0]:
f, ax = plt.subplots(1, 1, figsize=(12, 4), dpi=200)

ax.plot(model.wave, model.flux, lw=0.1)
ax.plot(cwave, cflux, lw=1)

#ax.set_xlim(3000, 4500)
#ax.set_xlim(6000, 7500)

# Using the PCA-trick

In [0]:
from pfs.ga.pfsspec.core.psf import PcaPsf

In [0]:
vdisp = VelocityDispersion(vdisp=1000, reuse_kernel=True)
s = vdisp.get_optimal_size(model.wave)
s

In [0]:
k, idx, shift = vdisp.eval_kernel(model.wave, size=s, normalize=True)
k.shape, idx.shape, shift

In [0]:
pca = PcaPsf.from_psf(vdisp, model.wave, size=s, normalize=True, truncate=200)
pca.mean.shape, pca.eigs.shape, pca.pc.shape

In [0]:
f, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=120)

ax.plot(np.log10(np.arange(pca.eigs.size) + 1), np.log10(pca.eigs))
ax.set_xlabel('log10 order')
ax.set_xlabel('log10 eigenvalue')
ax.set_title('Eigenvalue spectrum of PCA on kernels')

In [0]:
awave, aflux, _, shift = pca.convolve(model.wave, model.flux)
awave.shape, aflux.shape, shift, s

In [0]:
f, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=200)

ax.plot(model.wave, model.flux, lw=0.1)
ax.plot(cwave, cflux, lw=1)
ax.plot(awave, aflux, lw=1)

#ax.set_xlim(3000, 4500)
#ax.set_xlim(6000, 7500)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(6, 3), dpi=120)

ax.plot(cwave, (cflux - aflux) / cflux, lw=0.1)

#ax.set_xlim(3000, 4500)
#ax.set_xlim(6000, 7500)

ax.set_xlabel('wavelength [A]')
ax.set_ylabel('relative error of convolution')
ax.set_title(f'relative difference of PCA and direct convolution, k={pca.eigs.size}')

In [0]:
cflux - aflux